In [1]:
# !pip install langchain openai
!pip install "wandb>=0.15.3" -qqq
!pip install "langchain>=0.0.188" openai
!pip install transformers
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.7 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.


In [2]:
# from transformers import AutoTokenizer, AutoModelWithLMHead

In [2]:
import json
from pathlib import Path
from pprint import pprint

#### ChatGPT

In [3]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
# from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

# openai_api_key = 'sk-2Uk6tYcUnSzzVIn5M1ziT3BlbkFJpAWGTrQBsUAV6hLT34E'
# llm = OpenAI(temperature=0.0, openai_api_key=openai_api_key)

#### Falcon LLM

In [38]:
# import torch
# from transformers import BitsAndBytesConfig

# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=True,
# )

# model_id = "vilsonrodrigues/falcon-7b-instruct-sharded"

# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# model_4bit = AutoModelForCausalLM.from_pretrained(
#         model_id,
#         device_map="auto",
#         quantization_config=quantization_config,
#         trust_remote_code=True)
# tokenizer = AutoTokenizer.from_pretrained(model_id)

# import torch
# import transformers

# pipeline = transformers.pipeline(
#         "text-generation",
#         model=model_4bit,
#         tokenizer=tokenizer,
#         use_cache=True,
#         device_map="auto",
#         max_length=1024,
#         do_sample=True,
#         top_k=10,
#         num_return_sequences=1,
#         eos_token_id=tokenizer.eos_token_id,
#         pad_token_id=tokenizer.eos_token_id,
# )

# sequences = pipeline("tell me a joke")

# import locale
# from langchain import HuggingFacePipeline
# locale.getpreferredencoding = lambda: "UTF-8"

In [39]:
# API based on OpenAI API key
# API based on local LLM model eg, Falcon_13B, GPT_3.5_turbo, Llama_2

#### GPT4ALL

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

# pipeline = pipeline("text-generation", model="jondurbin/airoboros-gpt-3.5-turbo-100k-7b")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [ ]:
pipeline = pipeline("text-generation", model="TheBloke/Llama-2-7B-Chat-GGML")

In [41]:
llm = HuggingFacePipeline(pipeline=pipeline)

### Preparing data

In [13]:
def data_prep_chain1(query):
  app_json = '''{
          "formName": "Import Test App",
          "versionNumber": 1,
          "typeId": 1,
          "formDescription": "Import Test App",
          "language": [
            {
              "id": 3,
              "language": "English (India)",
              "nativeText": "English",
              "renderingOrder": "Left to right",
              "iso": "en",
              "renderingCode": "LTR",
              "lcid": "en",
              "isActive": 0,
              "isCustom": 0,
              "flag": true,
              "default": true
            }
          ],
          "icon": "",
          "formViewName": "View",
          "headerName": "Header",
          "workspaceId": [1, 203]
        }
      '''

  app_json = json.loads(app_json)

  # query = user_input + ". " + prompt_input

  custom_keys = '''
  {
    "formName": "Name of the application as specified by the user. If user didn't specify a name, Generate and assign it a name based on the 'user_input'.",
    "typeId": "This is a static integer type, you can assign it a value of 1.",
    "formDescription": "In about 5 to 10 words, describe what the application is about. Keep it short and to the point.",
    "formViewName": "Name of the view as specified by the user. If user didn't specify a name, give it a random name.",
    "headerName": "Name of the header as specified by the user. If user didn't specify a name, give it a random name.",
    "user_input": ""
  }
  '''

  custom_keys_json = json.loads(custom_keys)
  custom_keys_json["user_input"] = query

  json_pop = {}
  json_pop['app_json'] = app_json
  json_pop['custom_keys_json'] = custom_keys_json

  return json_pop

### Chain 1: for defining instructions on how to extract info from user input.

In [14]:
def memory_chain(data):
  template = """I am providing you with a JSON structure where you need to focus on the elements of the key 'custom_keys_json' the value of each key is a description of how to extract the actual value of the respective key.
  The structure also contains a key named 'user_input'.
  Your task is to return the keys (except the key named 'user_input') and the description of how each key can be extracted from a text. Also return the "user_input".

  % JSON STRUCTURE TO FOLLOW
  {app_json}

  YOUR RESPONSE:
  """

  prompt_template = PromptTemplate(input_variables=["app_json"], template=template)

  # Holds my 'location' chain
  location_chain = LLMChain(llm=llm, prompt=prompt_template)
  overall_chain = SimpleSequentialChain(chains=[location_chain], verbose=True)
  json_pop_str = str(data)
  review = overall_chain.run(json_pop_str)

### Chain 1.5: for how to convert the information to JSON for API

In [15]:
def form_chain(json_pop):
  template = """Your job is to extract the values of each key from the user_input key.
  Next, use 'app_json' JSON as a template and fill the value of keys.
  Next, assign the extracted value to each key.
  Finally return the created JSON structure.
  Note that the returned JSON must contain only the keys present in the 'app_json' elements.

  % USER INPUT
  {user_input}
  YOUR RESPONSE:
  """
  prompt_template = PromptTemplate(input_variables=["user_input"], template=template)

  instructions_chain = LLMChain(llm=llm, prompt=prompt_template)

  overall_chain = SimpleSequentialChain(chains=[instructions_chain], verbose=True)
  json_pop_str = str(json_pop)
  review = overall_chain.run(json_pop_str)

  return review

### API post for chain 1 (creating form).

In [16]:
import requests

def create_form(review, auth):
  app = json.loads(review)
  app['language'] = "[{\"id\":3,\"language\":\"English  (India)\",\"nativeText\":\"English\",\"renderingOrder\":\"Left to right\",\"iso\":\"en\",\"renderingCode\":\"LTR\",\"lcid\":\"en\",\"isActive\":0,\"isCustom\":0,\"flag\":true,\"default\":true}]"
  data = {'app': app}

  url = 'https://ai.sandbox.appsteer.io/services/web/form'

  form_data = {'app': json.dumps(data['app'])}

  headers = {"Authorization": "Bearer "+auth}

  print("creating form:", app)
  resp = requests.post(url=url, files=form_data, headers=headers)
  print(resp)
  print(resp.text)
  print('-'*50)
  return resp

### Chain 2: for getting fields from the model

In [17]:
def generate_page_data(data):
  app_name = json.loads(data)['formName']
  user_input = f"what all fields are required for a {app_name}"

  template = """Your task is to suggest 10-15 fields required for an application based on the user input provided.\
  Additionally, you need to assign relevant components from the given list to each suggested field. Please follow these instructions:
  1. Group the generated fields into different pages. Each group represents a page of the application and each page must have atmost 6 fields.
  2. Assign a header name to each page.
  3. Return the pages along with their header names and the fields included in them.
  Use the following list of components to assign to the fields:
  ["Text", "Numeric Text Box", "Label", "Multi select", "Dropdown List", "Radio Button", "Checkbox", "Separator", "Date Picker",
  "Variable", "Static Image", "Photo Validator", "Photograph", "Attachments", "Audio", "Video", "Signature"]

  % USER INPUT
  {user_input}
  YOUR RESPONSE:
  """

  prompt_template = PromptTemplate(input_variables=["user_input"], template=template)

  field_suggestion_chain = LLMChain(llm=llm, prompt=prompt_template)

  overall_chain = SimpleSequentialChain(chains=[field_suggestion_chain], verbose=True)
  review = overall_chain.run(user_input)

  return review

In [18]:
# output_fields = generate_page_data(output_ch1)

### Chain 2: Creating View and Headers

In [19]:
# Function creates data for LLM that would generate view and header JSON payloads.
def data_prep_chain2(prompt_input):
  view_json = '''{
  "commentRequired": 0,
  "formId": 797,
  "formViewName": "View 2",
  "generateNotifications": 0,
  "isMandatory": 1,
  "versionNumber": ""
  }
  '''

  view_json = json.loads(view_json)

  header_json = '''{
      "displayInView": 1,
      "formViewId": 1319,
      "headerAttributes": [],
      "headerName": "Header 1",
      "isMandatory": 1,
      "splitCount": 1,
      "versionNumber": ""
      }
      '''

  header_json = json.loads(header_json)

  json_pop = {}
  json_pop['view_json'] = view_json
  json_pop['header_json'] = header_json
  json_pop["user_input"] = prompt_input

  return json_pop

In [20]:
# Function for LLM that generates view and header JSON payloads.
def view_header_chain(json_pop):
  template = """Your job is to go through the text provided in 'user_input'.
  Next, you need to create 2 JSON objects using 'header_json' and 'view_json' as templates.
  Next, in header_json template, assign the value of 'headerName' as the value of 'Header Name' key from 'user_input'.
  Next, in view_json template assign the value of 'formViewName' as a page name (eg. Page 1) based on the 'user_input'.
  Finally, return a single JSON object containing the 2 created JSON objects.

  % USER INPUT
  {user_input}
  YOUR RESPONSE:
  """
  prompt_template = PromptTemplate(input_variables=["user_input"], template=template)

  field_chain = LLMChain(llm=llm, prompt=prompt_template)
  overall_chain = SimpleSequentialChain(chains=[field_chain], verbose=True)
  json_pop_str = str(json_pop)
  review = overall_chain.run(json_pop_str)

  return review

In [21]:
# Function POSTs view and header JSON payloads API.
def create_view(input, formId, auth):

  input = json.loads(input)
  view_json = input['view_json']
  view_json['formId'] = formId
  view_attributes = [{"attributeId":1,"attributeName":"Orientation","defaultAttributeValues":"[{\\\"id\\\":1,\\\"value\\\":\\\"Landscape\\\"},{\\\"id\\\":0,\\\"value\\\":\\\"Portrait\\\"}]","attributeValues":"{\\\"id\\\":0,\\\"value\\\":\\\"Portrait\\\"}","valueType":"JSON"},{"attributeId":2,"attributeName":"Orientation Lock","defaultAttributeValues":"false","attributeValues":"false","valueType":"Boolean"},{"attributeId":3,"attributeName":"Show Welcome Message","defaultAttributeValues":"false","attributeValues":"false","valueType":"Boolean"},{"attributeId":4,"attributeName":"Welcome Message","defaultAttributeValues":"<html><body><center><div><p style='color:#146a93; font-size:125%; font-family:roboto, sans-serif'>Select <B>Continue</B> to capture data<BR />or<BR /><B>Skip</B> if you don't want to capture data for this form</p></div></center></body></html>","attributeValues":"<html><body><center><div><p style='color:#146a93; font-size:125%; font-family:roboto, sans-serif'>Select <B>Continue</B> to capture data<BR />or<BR /><B>Skip</B> if you don't want to capture data for this form</p></div></center></body></html>","valueType":"String"},{"attributeId":5,"attributeName":"Capture Geo Location","defaultAttributeValues":"false","attributeValues":"false","valueType":"Boolean"},{"attributeId":6,"attributeName":"Time gap between two views(In Mins)","defaultAttributeValues":"0","attributeValues":"0","valueType":"Integer"},{"attributeId":7,"attributeName":"Show notification when view enabled","defaultAttributeValues":"false","attributeValues":"false","valueType":"Boolean"},{"attributeId":8,"attributeName":"Show Full Screen","defaultAttributeValues":"false","attributeValues":"false","valueType":"Boolean"},{"attributeId":9,"attributeName":"IS PII","defaultAttributeValues":"false","attributeValues":"false","valueType":"Boolean"},{"attributeId":10,"attributeName":"Restrict User/Group","defaultAttributeValues":"[]","attributeValues":"[]","valueType":"JSON"},{"attributeId":11,"attributeName":"IsDefault ","defaultAttributeValues":"false","attributeValues":"false","valueType":"Boolean"},{"attributeId":12,"attributeName":"Show Headers","defaultAttributeValues":"true","attributeValues":"true","valueType":"Boolean"}]
  view_json['viewAttribute'] = view_attributes

  url = 'https://ai.sandbox.appsteer.io/services/web/formView/CreateFormView'

  headers = {"Authorization": "Bearer "+auth}
  # post view
  print('posting view')
  print('view')
  # pprint(view_json)
  resp = requests.post(url=url, json=view_json, headers=headers)
  print(resp.text)
  print('-'*30)
  return resp

In [22]:
# Function POSTs view and header JSON payloads API.
def create_header(input, formViewId, auth):

  input = json.loads(input)
  header_json = input['header_json']
  header_json['formViewId'] = formViewId
  header_attributes = [{"headerId":0,"attributeId":1,"attributeName":"Restrict User/Group","defaultAttributeValues":"[]","attributeValues":"[]","valueType":"JSON","propertyBoxBehaviour":0},{"headerId":0,"attributeId":2,"attributeName":"Column Layout","defaultAttributeValues":"[{\\\"id\\\":0,\\\"value\\\":\\\"Responsive\\\"}]","attributeValues":"[{\\\"id\\\":0,\\\"value\\\":\\\"Responsive\\\"}]","valueType":"JSON","propertyBoxBehaviour":0},{"headerId":0,"attributeId":3,"attributeName":"Responsive","defaultAttributeValues":"[]","attributeValues":"[]","valueType":"JSON","propertyBoxBehaviour":0},{"headerId":0,"attributeId":4,"attributeName":"IsDefault ","defaultAttributeValues":"false","attributeValues":"false","valueType":"Boolean","propertyBoxBehaviour":0}]
  header_json['headerAttributes'] = header_attributes

  url = 'https://ai.sandbox.appsteer.io/services/web/header/CreateHeader'

  headers = {"Authorization": "Bearer "+auth}

  # post header
  print('posting header')
  print('header')
  # pprint(header_json)
  resp = requests.post(url=url, json=header_json, headers=headers)
  print(resp.text)
  print('-'*30)
  return resp

In [ ]:
# data_slice = output_fields.split("\n  \n  ")[0]
# data_output_ch2 = data_prep_chain2(data_slice)
# output_view_header_ch = view_header_chain(data_output_ch2)

In [ ]:
# create_view(output_view_header_ch, formId=798)
# create_header(output_view_header_ch, formViewId=1322)

### Chain 3: Creating fields

In [24]:
# Data preparation for creating fields JSON
def data_prep_chain3(prompt_input):
  fields_json = '''{
    "fieldAttributes": [{}],
    "headerId": 2575,
    "isMandatory": 1,
    "uiLabel": "Text",
    "uiType": 0,
    "validatorId": []
  }
  '''

  fields_json = json.loads(fields_json)

  field_ids = '''
      {
        "Text": 0,
        "Numeric Text Box": 18,
        "Label": 33,
        "Multi select": 31,
        "Dropdown List": 2,
        "Radio Button": 1,
        "Checkbox": 3,
        "Separator": 16,
        "Date Picker": 7,
        "variable": 38,
        "Static Image": 26,
        "Photo Validator": 23,
        "Photograph": 5,
        "Attachments": 21,
        "Audio": 8,
        "Video": 10,
        "Signature": 19
      }
  '''

  field_ids_json = json.loads(field_ids)
  field_ids_json['headerId'] = 2575

  json_pop = {}
  json_pop['fields_json'] = fields_json
  json_pop['field_ids_json'] = field_ids_json
  json_pop["user_input"] = prompt_input

  return json_pop

In [25]:
def field_chain(json_pop):
  template = """Your job is to go through the text provided in 'user_input', and for the listings under 'Fields', map what all keys from 'field_ids_json' are required.
  Now, the listings under 'Fields' in 'user_input' are in such a way, that the left part corresponds to the 'uiLabel' and the right part corresponds to the type of field (to be mapped).
  Next, for each of the required components, create a JSON structure with all the keys present in the 'fields_json'.
    Now in the created 'fields_json' object:
    1. assign the value of 'uiLabel' based on the extracted uiLabel from listings under Fields.
    2. assign the value of 'uiType' as an integer type based on the mapping provided in field_ids_json.

  Finally, concatenate the json objects of all the required components and return it
  Note that the returned JSONs must contain only the keys ['uiLabel', 'uiType']!

  % USER INPUT
  {user_input}
  YOUR RESPONSE:
  """
  prompt_template = PromptTemplate(input_variables=["user_input"], template=template)

  # Holds my 'location' chain
  field_chain = LLMChain(llm=llm, prompt=prompt_template)
  overall_chain = SimpleSequentialChain(chains=[field_chain], verbose=True)
  json_pop_str = str(json_pop)
  review = overall_chain.run(json_pop_str)

  return review

### API post for chain 3 (creating fields).

In [26]:
def create_fields(output_ch3, headerId, auth):

  field_template = '''{\
  "fieldAttributes": [{}],\
  "headerId": 123,\
  "isMandatory": 1,\
  "uiLabel": "Text",\
  "uiType": 0,\
  "validatorId": []\
  }\
  '''

  field_template = json.loads(field_template)
  field_template['headerId'] = headerId

  fields = json.loads(output_ch3)
  for field in fields:
    field_template['uiLabel'] = field['uiLabel']
    field_template['uiType'] = field['uiType']

    print('creating:', field_template)

    url = 'https://ai.sandbox.appsteer.io/services/web/formField/CreateFormField'

    headers = {"Authorization": "Bearer "+auth}

    resp = requests.post(url=url, json=field_template, headers=headers)
    print(resp.text)
    print('-'*30)

In [27]:
# output_ch3 = data_prep_chain3(data_slice)
# output_field_ch = field_chain(output_ch3)

In [ ]:
# create_fields(output_field_ch, headerId=2579)

### Testing the functions

##### chain 1 (creating a form)

In [28]:
query = "create a patient onboarding application"
auth = "eyJhbGciOiJIUzI1NiJ9.eyJ0ZW5hbnRVVUlEIjoiZjJjOTVmNzctMmVkZC00Mjk5LWI3ZmMtNDg5NjdhNDNiNTE2Iiwic3ViIjoiYWRtaW5AYXBwc3RlZXIuYWkiLCJyb2xlIjoiQ09SUE9SQVRFX0FETUlOIiwiaXNzIjoiQXBwU3RlZXIiLCJmdWxsTmFtZSI6IkFwcHN0ZWVyIEFJICIsIm9yZ0RvbWFpbk5hbWUiOiJhaS5hcHBzdGVlci5pbyIsInByZWZlcnJlZF91c2VybmFtZSI6ImFkbWluQGFwcHN0ZWVyLmFpIiwicm9sZVVVSUQiOiIzOWQwNjMwMC0xZjA3LTExZWUtOGVmZi0wMDIyNDgyNjk0ODAiLCJ1dWlkIjoiZDFkM2NkM2ItMWYwNy0xMWVlLThlZmYtMDAyMjQ4MjY5NDgwIiwidXNlcklkIjo3NDMsInVpZCI6ImRjNWMyYTk5LTQ1MTctNGQwYi05YWI5LTNiZDNmODhjZTY2OCIsInRlbmFudElkIjo0MSwiZXhwIjoxNjkwMTA1MzE1LCJpYXQiOjE2OTAwMTg5MTV9.SuvZCtcgoWSYeZixcLLs7oLFOZ1v_GK78Y2DrWz4ieI"

In [29]:
data = data_prep_chain1(query)

In [42]:
memory_chain(data)





> Entering new SimpleSequentialChain chain...



<p>The keys to be returned are the names of the keys specified in the JSON structure. The descriptions of each key can also be included in the response for the JSON structure.</p>

<p>You should also include the key named 'user_input' along with the description as it is a required field. </p>



> Finished chain.


In [43]:
output_ch1 = form_chain(data)





> Entering new SimpleSequentialChain chain...

{"formName": {"key":"app_json"}}

  The JSON structure that you need to generate should contain the values of each key specified in the JSON data. You can use a programming language like Python or JavaScript to extract and assign the values to your keys.



> Finished chain.


In [ ]:
create_form(output_ch1, auth)

creating form: {'formName': 'Patient Onboarding App', 'versionNumber': 1, 'typeId': 1, 'formDescription': 'Create a patient onboarding application', 'language': '[{"id":3,"language":"English  (India)","nativeText":"English","renderingOrder":"Left to right","iso":"en","renderingCode":"LTR","lcid":"en","isActive":0,"isCustom":0,"flag":true,"default":true}]', 'icon': '', 'formViewName': 'View', 'headerName': 'Header', 'workspaceId': [1, 203]}

<Response [200]>

{"code":1,"message":"App Created Successfully","addlInfo":"c0268b4d-c4c3-4843-b9ab-b7037e065b7e","data":{"formId":905,"formViewId":1459,"headerId":2674}}

--------------------------------------------------


<Response [200]>

##### chain 2 (creating view and header)

In [ ]:
print(output_ch1)



  {

    "formName": "Patient Onboarding App",

    "versionNumber": 1,

    "typeId": 1,

    "formDescription": "Create a patient onboarding application",

    "language": [

      {

        "id": 3,

        "language": "English (India)",

        "nativeText": "English",

        "renderingOrder": "Left to right",

        "iso": "en",

        "renderingCode": "LTR",

        "lcid": "en",

        "isActive": 0,

        "isCustom": 0,

        "flag": true,

        "default": true

      }

    ],

    "icon": "",

    "formViewName": "View",

    "headerName": "Header",

    "workspaceId": [

      1,

      203

    ]

  }


In [ ]:
output_fields = generate_page_data(output_ch1)





> Entering new SimpleSequentialChain chain...



  Page 1: Patient Information

  Header Name: Patient Information

  Fields: 

  1. Name - Text

  2. Age - Numeric Text Box

  3. Gender - Dropdown List

  4. Address - Text

  5. Phone Number - Numeric Text Box

  6. Email - Text

  

  Page 2: Medical History

  Header Name: Medical History

  Fields: 

  1. Medical Conditions - Multi select

  2. Allergies - Multi select

  3. Medications - Multi select

  4. Vaccinations - Multi select

  5. Family History - Text

  6. Smoking Status - Radio Button

  

  Page 3: Insurance Information

  Header Name: Insurance Information

  Fields: 

  1. Insurance Provider - Text

  2. Policy Number - Numeric Text Box

  3. Policy Expiry Date - Date Picker

  4. Co-Payment - Numeric Text Box

  5. Deductible - Numeric Text Box

  6. Copies of Insurance Card - Attachments



> Finished chain.


In [ ]:
data_slice = output_fields.split("\n  \n  ")[0]
print(data_slice)



  Page 1: Patient Information

  Header Name: Patient Information

  Fields: 

  1. Name - Text

  2. Age - Numeric Text Box

  3. Gender - Dropdown List

  4. Address - Text

  5. Phone Number - Numeric Text Box

  6. Email - Text


In [ ]:
data_output_ch2 = data_prep_chain2(data_slice)

In [ ]:
output_view_header_ch = view_header_chain(data_output_ch2)





> Entering new SimpleSequentialChain chain...



  {

  "header_json": {

    "displayInView": 1,

    "formViewId": 1319,

    "headerAttributes": [],

    "headerName": "Patient Information",

    "isMandatory": 1,

    "splitCount": 1,

    "versionNumber": ""

  },

  "view_json": {

    "commentRequired": 0,

    "formId": 797,

    "formViewName": "Page 1",

    "generateNotifications": 0,

    "isMandatory": 1,

    "versionNumber": ""

  }

}



> Finished chain.


In [ ]:
form_id = json.loads(x.text)['data']

NameError: ignored

In [ ]:
view_id = create_view(output_view_header_ch, formId=form_id, auth=auth)

In [ ]:
# (view_id.text)

In [ ]:
create_header(output_view_header_ch, formViewId=1381, auth=auth)

##### Chain 3 (creating fields)

In [ ]:
output_ch3 = data_prep_chain3(data_slice)

In [ ]:
output_ch3

{'fields_json': {'fieldAttributes': [{}],
  'headerId': 2575,
  'isMandatory': 1,
  'uiLabel': 'Text',
  'uiType': 0,
  'validatorId': []},
 'field_ids_json': {'Text': 0,
  'Numeric Text Box': 18,
  'Label': 33,
  'Multi select': 31,
  'Dropdown List': 2,
  'Radio Button': 1,
  'Checkbox': 3,
  'Separator': 16,
  'Date Picker': 7,
  'variable': 38,
  'Static Image': 26,
  'Photo Validator': 23,
  'Photograph': 5,
  'Attachments': 21,
  'Audio': 8,
  'Video': 10,
  'Signature': 19,
  'headerId': 2575},
 'user_input': '\n  Page 1: Patient Information\n  Header Name: Patient Information\n  Fields: \n  1. Name - Text\n  2. Age - Numeric Text Box\n  3. Gender - Dropdown List\n  4. Address - Text\n  5. Phone Number - Numeric Text Box\n  6. Email - Text'}

In [ ]:
output_field_ch = field_chain(output_ch3)





> Entering new SimpleSequentialChain chain...



  [

    {

      "uiLabel": "Name",

      "uiType": 0

    },

    {

      "uiLabel": "Age",

      "uiType": 18

    },

    {

      "uiLabel": "Gender",

      "uiType": 2

    },

    {

      "uiLabel": "Address",

      "uiType": 0

    },

    {

      "uiLabel": "Phone Number",

      "uiType": 18

    },

    {

      "uiLabel": "Email",

      "uiType": 0

    }

  ]



> Finished chain.


In [ ]:
create_fields(output_field_ch, headerId=2598, auth=auth)

### Auto generative function

In [ ]:
def auto_generative(query, auth):
  data = data_prep_chain1(query)
  memory_chain(data)
  output_ch1 = form_chain(data)
  response_form = create_form(output_ch1, auth)
  form_id = json.loads(response_form.text)['data']['formId']
  output_fields = generate_page_data(output_ch1)
  for data_slice in output_fields.split("\n  \n  "):
    data_output_ch2 = data_prep_chain2(data_slice)
    output_view_header_ch = view_header_chain(data_output_ch2)
    response_view = create_view(output_view_header_ch, formId=form_id, auth=auth)
    view_id = json.loads(response_view.text)['data']
    response_header = create_header(output_view_header_ch, formViewId=view_id, auth=auth)
    header_id = json.loads(response_header.text)['data']
    output_ch3 = data_prep_chain3(data_slice)
    output_field_ch = field_chain(output_ch3)
    create_fields(output_field_ch, headerId=header_id, auth=auth)

In [ ]:
query = "create a vendor onboarding application"
auth = "eyJhbGciOiJIUzI1NiJ9.eyJ0ZW5hbnRVVUlEIjoiZjJjOTVmNzctMmVkZC00Mjk5LWI3ZmMtNDg5NjdhNDNiNTE2Iiwic3ViIjoiYWRtaW5AYXBwc3RlZXIuYWkiLCJyb2xlIjoiQ09SUE9SQVRFX0FETUlOIiwiaXNzIjoiQXBwU3RlZXIiLCJmdWxsTmFtZSI6IkFwcHN0ZWVyIEFJICIsIm9yZ0RvbWFpbk5hbWUiOiJhaS5hcHBzdGVlci5pbyIsInByZWZlcnJlZF91c2VybmFtZSI6ImFkbWluQGFwcHN0ZWVyLmFpIiwicm9sZVVVSUQiOiIzOWQwNjMwMC0xZjA3LTExZWUtOGVmZi0wMDIyNDgyNjk0ODAiLCJ1dWlkIjoiZDFkM2NkM2ItMWYwNy0xMWVlLThlZmYtMDAyMjQ4MjY5NDgwIiwidXNlcklkIjo3NDMsInVpZCI6ImY2NTkwMTgwLWExMDgtNGE5ZS1hNzNkLTU1ZGMzY2M2MmI3YSIsInRlbmFudElkIjo0MSwiZXhwIjoxNjkwMjkxNTA4LCJpYXQiOjE2OTAyMDUxMDh9.PFLmy9vbEk1p8TBsNLQsQVGIbs3g0Eh8j3csZZnmiFQ"

In [ ]:
auto_generative(query, auth)





> Entering new SimpleSequentialChain chain...



  {

  'formName': "Name of the application as specified by the user. If user didn't specify a name, Generate and assign it a name based on the 'user_input'.", 

  'typeId': 'This is a static integer type, you can assign it a value of 1.', 

  'formDescription': 'In about 5 to 10 words, describe what the application is about. Keep it short and to the point.', 

  'formViewName': "Name of the view as specified by the user. If user didn't specify a name, give it a random name.", 

  'headerName': "Name of the header as specified by the user. If user didn't specify a name, give it a random name.", 

  'user_input': 'create a vendor onboarding application'

  }



> Finished chain.





> Entering new SimpleSequentialChain chain...



  {

    "formName": "Vendor Onboarding Application",

    "versionNumber": 1,

    "typeId": 1,

    "formDescription": "Create a vendor onboarding application",

    "language": [

      {

        "id": 